# Wrangle 
Data Acquisition + Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

import warnings
warnings.filterwarnings("ignore")

from wrangle import prep_fitbit, split_fitbit, visualize_split

In [2]:
# wrangled data 
df = prep_fitbit()

In [3]:
# drop categorical columns (only useful for visuals)
df.drop(columns=['month', 'weekday'], inplace=True)

In [4]:
df.head()

,calories_burned,steps,distance,floors,minutes_sedentary,minutes_lightly_active,minutes_fairly_active,minutes_very_active,activity_calories
date,,,,,,,,,
2018-04-26,2635,2931,1.38,2.0,1347,93.0,0.0,0.0,566
2018-04-27,3649,9971,4.70,19.0,1158,219.0,46.0,17.0,1752
2018-04-28,3379,3521,1.66,2.0,702,155.0,22.0,11.0,1124
2018-04-29,2851,1639,0.77,0.0,891,122.0,0.0,0.0,602
2018-04-30,2825,1323,0.62,0.0,880,104.0,0.0,0.0,540


In [5]:
# split data into train 50% validate 30% test 20%
train, validate, test = split_fitbit(df)

---
# Modeling
> Forecasting: Predict the next two weeks of activity from 12/07/18 - 12/21/18